In [1]:
import nltk
#nltk.download()

In [2]:
import requests
import pandas as pd
import re, string, unicodedata
from pprint import pprint
from bs4 import BeautifulSoup
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer

#import contractions
#import inflect

In [3]:
MADRID_URL = "https://www.madridiario.es/sucesos"

In [4]:
def get_soup(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser") #parser para que no se queje
    return soup

soup = get_soup(MADRID_URL)

In [5]:
def get_headers(soup):
    news = []
    headers_container = soup.find_all("h2", {'class': 'titulo'})
    for header in headers_container:
        news.append(header.text)
    return news

news_raw = get_headers(soup) #return dataframe
pprint(news_raw[:6])

['Un motorista, herido grave mientras circulaba por el Circuito del Jarama',
 'Una nave industrial, arrasada por un incendio en Alcalá de Henares',
 'Un paracaidista cae a un tejado en el encuentro de veteranos de la Base '
 'Príncipe de Paracuellos',
 'Un accidente entre un coche y una moto se salda con un herido grave en Parla',
 'Un hombre sufre varios traumatismos graves al caer con su bicicleta en la '
 'carretera',
 'Desarticulado un grupo criminal que se dedicaba al robo de coches para su '
 'posterior venta']


In [6]:
# Get rid of lower-cases
def tolower (work_list):
    return [row.lower() for row in work_list]
    
news_raw = tolower(news_raw)
pprint(news_raw[:6])

['un motorista, herido grave mientras circulaba por el circuito del jarama',
 'una nave industrial, arrasada por un incendio en alcalá de henares',
 'un paracaidista cae a un tejado en el encuentro de veteranos de la base '
 'príncipe de paracuellos',
 'un accidente entre un coche y una moto se salda con un herido grave en parla',
 'un hombre sufre varios traumatismos graves al caer con su bicicleta en la '
 'carretera',
 'desarticulado un grupo criminal que se dedicaba al robo de coches para su '
 'posterior venta']


In [7]:
# Get rid of accents
def clean_accent(work_list):
    return [row.replace('á','a').replace('é','e').replace('í','i').replace('ó','o').replace('ú','u') for row in work_list]

news_raw2 = clean_accent(news_raw)
pprint(news_raw2[:6])

['un motorista, herido grave mientras circulaba por el circuito del jarama',
 'una nave industrial, arrasada por un incendio en alcala de henares',
 'un paracaidista cae a un tejado en el encuentro de veteranos de la base '
 'principe de paracuellos',
 'un accidente entre un coche y una moto se salda con un herido grave en parla',
 'un hombre sufre varios traumatismos graves al caer con su bicicleta en la '
 'carretera',
 'desarticulado un grupo criminal que se dedicaba al robo de coches para su '
 'posterior venta']


In [8]:
#Get rid of commas
def clean_commas(work_list):
    return [row.replace(',','') for row in work_list]

dfnews_raw3 = clean_commas(news_raw2)
pprint(dfnews_raw3[:6])

['un motorista herido grave mientras circulaba por el circuito del jarama',
 'una nave industrial arrasada por un incendio en alcala de henares',
 'un paracaidista cae a un tejado en el encuentro de veteranos de la base '
 'principe de paracuellos',
 'un accidente entre un coche y una moto se salda con un herido grave en parla',
 'un hombre sufre varios traumatismos graves al caer con su bicicleta en la '
 'carretera',
 'desarticulado un grupo criminal que se dedicaba al robo de coches para su '
 'posterior venta']


In [9]:
# Get each word / Tokenizar
def token(work_list):
    return [nltk.word_tokenize(row) for row in work_list]

token_news = token(dfnews_raw3)
print(token_news[:2])

[['un', 'motorista', 'herido', 'grave', 'mientras', 'circulaba', 'por', 'el', 'circuito', 'del', 'jarama'], ['una', 'nave', 'industrial', 'arrasada', 'por', 'un', 'incendio', 'en', 'alcala', 'de', 'henares']]


In [10]:
stopWords = set(stopwords.words('spanish'))

In [11]:
print(stopWords)

{'estamos', 'serías', 'tendría', 'vuestra', 'hubieron', 'habrían', 'sea', 'vosostras', 'hayáis', 'sentida', 'está', 'fueran', 'míos', 'sobre', 'estuviesen', 'sois', 'y', 'hubiesen', 'tengáis', 'del', 'estada', 'habríamos', 'porque', 'hube', 'tuyos', 'es', 'tuya', 'estaréis', 'están', 'vosostros', 'hubiera', 'fuimos', 'hubieran', 'ese', 'éramos', 'ante', 'por', 'esta', 'tendré', 'tendríais', 'habíais', 'estuvieran', 'hubieras', 'nosotras', 'sentido', 'estáis', 'tendrán', 'habíamos', 'serás', 'esa', 'tened', 'mías', 'desde', 'estarán', 'estuviéramos', 'erais', 'hubiéramos', 'en', 'os', 'hubieseis', 'estuvisteis', 'tenía', 'tenemos', 'vuestro', 'ni', 'tus', 'tenían', 'hubo', 'hubieses', 'tuviste', 'estado', 'sean', 'tuviesen', 'contra', 'habremos', 'tenga', 'e', 'algunas', 'estaremos', 'hayas', 'habida', 'sentid', 'hubiste', 'unos', 'teníais', 'estar', 'esté', 'quienes', 'todo', 'tuvo', 'lo', 'estaríais', 'me', 'tendrías', 'que', 'otras', 'tendréis', 'estuvieseis', 'le', 'estarías', 'estu

In [14]:


def clean_stopwords(list_tokens, stp_words):
    [row.remove(item) for row in list_tokens for item in row if item in stp_words]
    return list_tokens
    
cleaned_data = clean_stopwords(token_news, stopWords)
pprint(cleaned_data)


[['motorista',
  'herido',
  'grave',
  'mientras',
  'circulaba',
  'circuito',
  'jarama'],
 ['nave', 'industrial', 'arrasada', 'incendio', 'alcala', 'henares'],
 ['paracaidista',
  'cae',
  'tejado',
  'encuentro',
  'veteranos',
  'base',
  'principe',
  'paracuellos'],
 ['accidente', 'coche', 'moto', 'salda', 'herido', 'grave', 'parla'],
 ['hombre',
  'sufre',
  'varios',
  'traumatismos',
  'graves',
  'caer',
  'bicicleta',
  'carretera'],
 ['desarticulado',
  'grupo',
  'criminal',
  'dedicaba',
  'robo',
  'coches',
  'posterior',
  'venta'],
 ['herido', 'grave', 'hombre', 'invidente', 'sordomudo', 'atropellado', 'moto'],
 ['sentencia',
  'supremo',
  "'la",
  'manada',
  "'",
  'reaviva',
  'debate',
  'reforma',
  'codigo',
  'penal'],
 ['localizado', 'argentino', '72', 'años', 'desaparecido', 'barajas'],
 ['mas',
  '130',
  'organizaciones',
  'feministas',
  'volveran',
  'entonar',
  "'yo",
  'si',
  'creo',
  "'"],
 ['fiscal', 'defiende', 'manada', 'cometio', 'violacion'